# Setup

In [ ]:
%pip install -e ..[dev,gpu]

# 🤖 Focoos Foundational Models


In [ ]:
import os
from pprint import pprint

from focoos import DEV_API_URL, Focoos

focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"), host_url=DEV_API_URL)

pprint(focoos.list_focoos_models())

## Cloud Inference

In [ ]:
import os
from pprint import pprint

from supervision import plot_image

from focoos import DEV_API_URL, Focoos

model_ref = "focoos_object365"
image_path = "./assets/ade_val_034.jpg"

focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"), host_url=DEV_API_URL)

model = focoos.get_remote_model(model_ref)
## Only admin can deploy foundational models

output, preview = model.infer(image_path, threshold=0.4, annotate=True)
plot_image(preview)

## Local Inference

This section demonstrates how to perform local inference using a model from the Focoos platform. 
We will load a model, deploy it locally, and then run inference on a sample image.


### Available Runtime Types

In [ ]:
from focoos.ports import RuntimeTypes

for runtime_type in RuntimeTypes:
    print(runtime_type)

### OnnxRuntime With CUDA (focoos_object365)

In [ ]:
import os
from pprint import pprint

from supervision import plot_image

from focoos import Focoos

focoos = Focoos(
    api_key=os.getenv("FOCOOS_API_KEY"),
)
image_path = "./assets/ade_val_034.jpg"
model_ref = "focoos_object365"


model = focoos.get_local_model(model_ref)

latency = model.benchmark(iterations=10, size=640)
pprint(latency)
# pprint(latency)
output, preview = model.infer(image_path, threshold=0.3, annotate=True)
pprint(output.detections)
pprint(output.latency)

plot_image(preview)

### OnnxRuntime With TensorRT (FP16) (focoos_object365)

In [ ]:
import os
from pprint import pprint

from supervision import plot_image

from focoos import Focoos
from focoos.ports import RuntimeTypes

focoos = Focoos(
    api_key=os.getenv("FOCOOS_API_KEY"),
)
image_path = "./assets/ade_val_034.jpg"
model_ref = "focoos_object365"


model = focoos.get_local_model(model_ref, runtime_type=RuntimeTypes.ONNX_TRT16)

latency = model.benchmark(iterations=10, size=640)
pprint(latency)
# pprint(latency)
output, preview = model.infer(image_path, threshold=0.3, annotate=True)
pprint(output.detections)
pprint(output.latency)

plot_image(preview)

# User Models

### List User Models

In [ ]:
import os
from pprint import pprint

from dotenv import load_dotenv

from focoos import DEV_API_URL, Focoos

load_dotenv()

focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"), host_url=DEV_API_URL)

models = focoos.list_models()
pprint(models)

## Create Model

In [ ]:
import os
from pprint import pprint

from dotenv import load_dotenv

from focoos import DEV_API_URL, Focoos

load_dotenv()

focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"), host_url=DEV_API_URL)

model = focoos.new_model(name="test-model", focoos_model="focoos_object365", description="Test model")
### Get Model Info

## Deploy user model on shared cloud endpoint

In [ ]:
import os
from pprint import pprint

from dotenv import load_dotenv

from focoos import DEV_API_URL, Focoos

load_dotenv()

focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"), host_url=DEV_API_URL)

model = focoos.get_remote_model("fa94df6806c84c11")
model_info = model.get_info()

## Cloud Inference

In [ ]:
import os
from pprint import pprint

import supervision as sv
from dotenv import load_dotenv

from focoos import Focoos

load_dotenv()

focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"), host_url=DEV_API_URL)

model = focoos.get_remote_model("fa94df6806c84c11")
model_info = model.get_info()
image_path = "./assets/aquarium.jpg"

output, preview = model.infer(image_path, threshold=0.5, annotate=True)
sv.plot_image(preview)

## Local Inference




In [ ]:
import os
from pprint import pprint

from supervision import plot_image

from focoos import DEV_API_URL, Focoos

focoos = Focoos(
    api_key=os.getenv("FOCOOS_API_KEY"),
    host_url=DEV_API_URL,
)
image_path = "./assets/aquarium.jpg"
model_ref = "focoos_object365"

model_info = focoos.get_model_info(model_ref)

model = focoos.get_local_model(model_ref)

latency = model.benchmark(iterations=10, size=640)
# pprint(latency)
output, preview = model.infer(image_path, threshold=0.3, annotate=True)

plot_image(preview)

## Train a Model

### List Public Datasets


In [ ]:
import os
from pprint import pprint

from dotenv import load_dotenv

from focoos import DEV_API_URL, Focoos

load_dotenv()

focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"), host_url=DEV_API_URL)

models = focoos.list_models()
pprint(models)

model = focoos.new_model(name="test-model-2", focoos_model="focoos_object365", description="Test model")


model = focoos.get_remote_model("fa94df6806c84c11")

datasets = focoos.list_shared_datasets()
pprint(datasets)

In [ ]:
from focoos.ports import Hyperparameters, TrainInstance

model = focoos.get_remote_model("fa94df6806c84c11")

res = model.train(
    anyma_version="0.11.1",
    dataset_ref="11e80dd77806450f",
    instance_type=TrainInstance.ML_G4DN_XLARGE,
    volume_size=50,
    max_runtime_in_seconds=36000,
    hyperparameters=Hyperparameters(
        learning_rate=0.0001,
        batch_size=16,
        max_iters=1500,
        eval_period=100,
        resolution=640,
    ),  # type: ignore
)
pprint(res)

In [ ]:
import time

from focoos.utils.logger import get_logger

completed_status = ["Completed", "Failed"]
logger = get_logger(__name__)
model = focoos.get_remote_model("fa94df6806c84c11")
status = model.train_status()
while status["main_status"] not in completed_status:
    status = model.train_status()
    logger.info(f"Training status: {status['main_status']}")
    pprint(f"Training progress: {status['status_transitions']}")
    time.sleep(30)

In [ ]:
logs = model.train_logs()
pprint(logs)

## Get User Info

In [ ]:
import os
from pprint import pprint

from focoos import DEV_API_URL, Focoos

focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"), host_url=DEV_API_URL)

user_info = focoos.get_user_info()
pprint(user_info.model_dump())

## Get System Info

In [ ]:
from focoos.utils.system import get_system_info

system_info = get_system_info()
system_info.pretty_print()